In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import requests
import wikipedia
import re
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen
from scrapy import selector
import datetime as dt
import pickle
from skimage import io
from IPython.display import clear_output
import pdb
plt.style.use('ggplot')
%matplotlib inline

In [2]:
odi = pickle.load(open('odi_list', "rb" ))
odi[1998].reset_index(inplace=True)
link = odi[1998].iloc[36,-1]
odi[1998]



,odi_index,team_1,team_2,winner,margin,ground,match_date,scorecard_link
0,odi # 1270,new zealand,south africa,south africa,2 runs,brisbane,"jan 9, 1998",http://stats.espncricinfo.com//ci/engine/match...
1,odi # 1271,bangladesh,india,india,4 wickets,dhaka,"jan 10, 1998",http://stats.espncricinfo.com//ci/engine/match...
2,odi # 1273,india,pakistan,india,18 runs,dhaka,"jan 11, 1998",http://stats.espncricinfo.com//ci/engine/match...
3,odi # 1272,australia,south africa,south africa,5 wickets,brisbane,"jan 11, 1998",http://stats.espncricinfo.com//ci/engine/match...
4,odi # 1274,bangladesh,pakistan,pakistan,9 wickets,dhaka,"jan 12, 1998",http://stats.espncricinfo.com//ci/engine/match...
...,...,...,...,...,...,...,...,...
103,odi # 1373,india,zimbabwe,zimbabwe,13 runs,sharjah,"nov 11, 1998",http://stats.espncricinfo.com//ci/engine/match...
104,odi # 1374,india,zimbabwe,india,10 wickets,sharjah,"nov 13, 1998",http://stats.espncricinfo.com//ci/engine/match...
105,odi # 1375,pakistan,zimbabwe,pakistan,4 wickets,gujranwala,"nov 20, 1998",http://stats.espncricinfo.com//ci/engine/match...
106,odi # 1376,pakistan,zimbabwe,zimbabwe,6 wickets,sheikhupura,"nov 22, 1998",http://stats.espncricinfo.com//ci/engine/match...


# Obtain Scoresheet

In [6]:
def get_squad(batting_sheet):
    """Gets team members, captain and keeper
    """
    team = []
    
    # batsmen who batted
    batsmen = batting_sheet.find_all('div', class_ = "wrap batsmen")
    for bat in batsmen:
        player = bat.find('div', class_ = "cell batsmen").text
        if player.find('(c)')>-1:
            captain = player.replace('(c)','').replace('†','').strip().lower()
        if player.find('†')>-1:
            keeper = player.replace('†','').replace('(c)','').strip().lower()
        player = player.replace('†','').replace('(c)','').strip()
        team.append(player.lower())
    did_not_bat = batting_sheet.find('div', class_ = "wrap dnb")
    
    
    # Now batsmen who did not bat
    for dnb in did_not_bat.find_all('a'):
        team.append(dnb.text.lower())
    return team, captain, keeper


def get_full_name(name, team):
    """Accepts last name and returns players name
    """
    name = name.lower()
    name = name.replace('†','').replace('(c)','')
    for member in team:
        if member.lower().find(name) > -1:
            return member
        
def get_innings(innings, teams):
    """Returns batting and bowling innings
    """
    
    for i,inning in enumerate(innings):
        for team in teams:
            if inning.find(team)>-1:
                innings[i] = team
    
    return innings
                

        
def get_toss_outcome(toss_string):
    pass

In [4]:

outs = ['b', 'c', 'lbw', 'st', 'hw', 'ro', 'hand', 'obs']

def how_out(dismissal, team):
    """Returns triplet tuple as (how, fielder, bowler)
    """
    
    dismissal = dismissal.lower().strip()
    
    if dismissal == 'not out':
        return 'not out', '', ''
    
    bowler = re.findall(r"b\s(.+)", dismissal)
    fielder = re.findall(r"c\s(.+)\sb", dismissal)
    if len(bowler)>0:
        bowler = bowler[0]
    if len(fielder)>0:
        fielder = fielder[0]
    
    # print(bowler, fielder)
    
    if len(fielder) == 0:
        keeper = re.findall(r"st\s†(.+)\sb", dismissal)
        if len(keeper)>0:
            keeper = keeper[0]
    
    # bowled
    if len(fielder) == 0 and dismissal == 'b ' + bowler:
        how = 'bowled'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # caught
    if len(fielder) > 0:
        how = 'caught'
        fielder = get_full_name(fielder, team)
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # lbw
    if dismissal.find('lbw')>-1 or dismissal.find('leg before')>-1:
        how = 'lbw'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # hit wicket
    if dismissal.find('hit wicket')>-1:
        how = 'hit wicket'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # run out
    if dismissal.find('run out')>-1:
        how = 'run out'
        fielder = ''
        bowler = ''
        return how, fielder, bowler
    
    # handled
    if dismissal.find('handled')>-1:
        how = 'handled the ball'
        fielder = ''
        bowler = ''
        return how, fielder, bowler

    # obstructing the fieldsman
    if dismissal.find('obstruct') > -1:
        how = 'obstruct'
        fielder = ''
        blower = ''
        return how, fielder, bowler
    
    
    # caught and bowled
    if dismissal.find('&') > -1:
        how = 'caught'
        bowler = get_full_name(bowler, team)
        fielder = bowler
        return how, fielder, bowler
    
    
    # Stumped
    if dismissal.find('st †') > -1:
        how = 'stumped'
        fielder = get_full_name(keeper, team)
        bowler = get_full_name(bowler, team)
        return 'stumped',fielder, bowler


In [5]:
how_out('st †Kaluwitharana b Muralitharan', team[1])

NameError: name 'team' is not defined

In [27]:
link = odi[1998].iloc[36,-1]
link
team_1 = odi[1998].iloc[36,:].team_1
team_2 = odi[1998].iloc[36,:].team_2
print(link)
team_1, team_2

http://stats.espncricinfo.com//ci/engine/match/66080.html


('south africa', 'sri lanka')

In [28]:
odi[1998].head()

,odi_index,team_1,team_2,winner,margin,ground,match_date,scorecard_link
0,odi # 1270,new zealand,south africa,south africa,2 runs,brisbane,"jan 9, 1998",http://stats.espncricinfo.com//ci/engine/match...
1,odi # 1271,bangladesh,india,india,4 wickets,dhaka,"jan 10, 1998",http://stats.espncricinfo.com//ci/engine/match...
2,odi # 1273,india,pakistan,india,18 runs,dhaka,"jan 11, 1998",http://stats.espncricinfo.com//ci/engine/match...
3,odi # 1272,australia,south africa,south africa,5 wickets,brisbane,"jan 11, 1998",http://stats.espncricinfo.com//ci/engine/match...
4,odi # 1274,bangladesh,pakistan,pakistan,9 wickets,dhaka,"jan 12, 1998",http://stats.espncricinfo.com//ci/engine/match...


In [103]:
# def get_scoresheet(link, format):

year = 1998
link = odi[year].iloc[36,-1]
link
matchID = odi[year].iloc[36,0]

if matchID.find('odi')>-1:
    match_format = 'odi'
elif matchID.find('test')>-1:
    match_format = 'test'

print("odi_index:", odi_index)
team_1 = odi[1998].iloc[36,:].team_1
team_2 = odi[1998].iloc[36,:].team_2
team_names = [team_1, team_2]
opponents = {}
opponents[team_1] = team_2
opponents[team_2] = team_1

# Get the link
# r = requests.get(link)
soup = BeautifulSoup(r.content,'lxml')

# Number of score sheets = number of innings
# Each scoresheet contains both batting and bowling information
scoresheets = soup.find_all('article', class_ = "sub-module scorecard")

# One 
game_information = soup.find('article', class_ = "sub-module game-information pre")

num_innings = len(scoresheets)

bat_innings = []
ball_innings = []

squad = {}
squad[team_1] = []
squad[team_2] = []

captain = {}
captain[team_1] = []
captain[team_2] = []

keeper = {}
keeper[team_1] = []
keeper[team_2] = []

# Get innings Info, team, captain, keeper
for i,sheet in enumerate(scoresheets):
    
    
    if match_format == 'odi':
        my_str = sheet.find('div', class_ = "accordion-header").text.lower().strip()
        this_innings = re.findall(r'(.+)\sinnings', my_str)[0]
    elif matc_format == 'test':
        my_str = sheet.find('div', class_ = "accordion-header").text.lower().strip()
        this_innings = re.findall(r'(.+)\s\d', my_str)[0]

    bat_innings.append(this_innings)

ball_innings = [opponents[x] for x in bat_innings]

# Get squad for team batting first
team_batting_first = bat_innings[0]
batting_sheet = scoresheets[0].find("div", class_="scorecard-section batsmen")
squad[team_batting_first], captain[team_batting_first], keeper[team_batting_first] = get_squad(batting_sheet)

# Get squad for team batting second
team_batting_second = bat_innings[1]
batting_sheet = scoresheets[1].find("div", class_="scorecard-section batsmen")
squad[team_batting_second], captain[team_batting_second], keeper[team_batting_second] = get_squad(batting_sheet)


#############################################################################################
    
# Now get batting and bowling info
BATTING = {}
for s,sheet in enumerate(scoresheets):
    
    batting_sheet = sheet.find("div", class_="scorecard-section batsmen")
    bowling_sheet = sheet.find("div", class_ = "scorecard-section bowling")
    inning = 'inning_' + str(s+1)
    
    batting_team = bat_innings[s]
    bowling_team = ball_innings[s]
    
    ## DEFINE HEADERS
    # get all the header.
    # Some games have SR and balles, some not, so important to get it on a game-by-game basis
    all_headers = []
    for header in batting_sheet.find('div', class_ = "wrap header").find_all('div'):
        all_headers.append(header.text.lower())
        
    # The commentary header is usually emptty: just add commentary
    if len(all_headers[1])==0:
        all_headers[1] = 'commentary'
        
    # Create dataframe
    all_headers =  all_headers + ['how', 'fielder', 'bowler'] + ['matchID', 'year', 'team', 'innings']
    score_card_batting = pd.DataFrame(columns = all_headers)
    
    # Iterate over batesmen
    all_batsmen  = batting_sheet.find_all('div', class_ = "wrap batsmen")
    for i, batsman in enumerate(all_batsmen):
        
        # iterate over cells
        for j,info in enumerate(batsman.find_all('div')):
            # print(j,info.text, type(info.text))
        
            if j == 0: #name
                player = info.text.lower().replace('(c)','').replace('†','').strip()
                score_card_batting.loc[i, all_headers[j]] = player
                continue
            if j == 1: #commentary
                # print(info.text.lower().strip())
                how,fielder,bowler = how_out(info.text.lower().strip(), squad[bowling_team])
                        
            
            score_card_batting.loc[i, 'how'] = how
            score_card_batting.loc[i, 'fielder'] = fielder
            score_card_batting.loc[i, 'bowler'] = bowler
            score_card_batting.loc[i, all_headers[j]] = info.text.lower().strip()
        
    did_not_bat = set(squad[batting_team]) - set(score_card_batting.batsmen.values)
    print(did_not_bat)
    
    l = len(score_card_batting)
    for dnb in did_not_bat:
        score_card_batting.loc[l, all_headers[0]] = dnb
        l = l+1
        
            
            
    # Now fill match-cells: common to all batsmen
    score_card_batting.loc[:, 'matchID'] = matchID
    score_card_batting.loc[:, 'year'] = year
    score_card_batting.loc[:, 'innings'] = s+1
    score_card_batting.loc[:, 'team'] = batting_team
    
    BATTING[inning] = score_card_batting.fillna(0)
            

#     ############## Bowling Info
    bowling_sheet = sheet.find("div", class_ = "scorecard-section bowling")
    


odi_index: odi # 1306
{'r telemachus'}
set()


In [105]:
BATTING['inning_2']

,batsmen,commentary,r,b,m,4s,6s,sr,,how,fielder,bowler,matchID,year,team,innings
0,st jayasuriya,c crookes b cronje,68,83,117,6,1,81.92,,caught,dn crookes,wj cronje,odi # 1306,1998,sri lanka,2
1,ms atapattu,c rhodes b telemachus,4,17,16,0,0,23.52,,caught,jn rhodes,r telemachus,odi # 1306,1998,sri lanka,2
2,rs mahanama,c kirsten b elworthy,11,19,32,1,0,57.89,,caught,g kirsten,s elworthy,odi # 1306,1998,sri lanka,2
3,pa de silva,c rindel b cronje,28,46,64,3,0,60.86,,caught,mjr rindel,wj cronje,odi # 1306,1998,sri lanka,2
4,rs kaluwitharana,c †boucher b telemachus,19,22,34,2,0,86.36,,caught,mv boucher,r telemachus,odi # 1306,1998,sri lanka,2
5,a ranatunga,c †boucher b crookes,0,4,3,0,0,0.00,,caught,mv boucher,dn crookes,odi # 1306,1998,sri lanka,2
6,hdpk dharmasena,c rindel b crookes,21,30,29,1,0,70.00,,caught,mjr rindel,dn crookes,odi # 1306,1998,sri lanka,2
7,udu chandana,c klusener b elworthy,26,36,50,1,0,72.22,,caught,l klusener,s elworthy,odi # 1306,1998,sri lanka,2
8,wpujc vaas,b telemachus,8,11,11,1,0,72.72,,bowled,,r telemachus,odi # 1306,1998,sri lanka,2
9,gp wickramasinghe,b telemachus,0,1,1,0,0,0.00,,bowled,,r telemachus,odi # 1306,1998,sri lanka,2
